In [9]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [10]:
df = pd.read_csv('data/engineered_grit_data.csv')
df.head()

,surveyelapse,VCL1,VCL2,VCL3,VCL4,VCL5,VCL7,VCL8,VCL10,VCL11,...,operatingsystem_ChromeOS,operatingsystem_Linux,operatingsystem_Macintosh,operatingsystem_Unknown,operatingsystem_Windows,operatingsystem_Windows Phone,operatingsystem_iOS,age_group_18-30,age_group_30-45,age_group_45+
0,340,0,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
1,126,1,1,1,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,1,0
2,130,1,1,1,1,1,0,1,1,1,...,0,0,0,0,1,0,0,0,0,1
3,592,1,0,0,1,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
4,217,1,1,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0


In [ ]:
X = df[['C8', 'C9', 'C1', 'C10', 'C5', 'C4', 'N7', 'C3', 'A10', 'C2', 'N5', 'A8', 'O2', 'E10', 'age_group_18-30']]
#X = df.drop(columns = ['Grit', 'highgrit'])
y = df['Grit']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test  = model.predict(X_test)

MAE_train = mean_absolute_error(y_train, y_pred_train)
MAE_test  = mean_absolute_error(y_test, y_pred_test)

RMSE_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
RMSE_test  = np.sqrt(mean_squared_error(y_test, y_pred_test))

R2_train = r2_score(y_train, y_pred_train)
R2_test  = r2_score(y_test, y_pred_test)

print("MAE_train:", MAE_train)
print("MAE_test:", MAE_test)

print("RMSE_train:", RMSE_train)
print("RMSE_test:", RMSE_test)

print("R2_train:", R2_train)
print("R2_test:", R2_test)

MAE_train: 0.20372409610585737
MAE_test: 0.3911044959769105
RMSE_train: 0.2564692278324161
RMSE_test: 0.49160260537538764
R2_train: 0.8656309813615554
R2_test: 0.490326127891772
